In [20]:
#importing the necessary libraries
import numpy as np
import pandas as pd
import warnings
warnings.warn('Make sure you are in the PyViz environment')
import plotly as plt
import os
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import requests

C:\Users\andre\anaconda3\envs\newviznow\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Make sure you are in the PyViz environment
  """


In [3]:
def load():
    #load dotenv in to call api keys with
    load_dotenv()

    #Load Alpaca keys
    warnings.warn('Make sure your api keys are renamed to fit this function or alter the function to fit your names')
    alpaca_public = os.getenv('ALPACA_API_KEY')
    alpaca_secret = os.getenv('ALPACA_SECRET_KEY')
    api = tradeapi.REST(alpaca_public, alpaca_secret, api_version='v2')

    return api
    

def data_pull(api):

    #initialize stock choice
    assets = api.list_assets()
    tick_list = []
    tickers = ['F', 'AAPL','ENPH','WMT','KO']
    #Try to call the historical quote of the stock
    for ticker in tickers:
        try:
            #print(ticker)
            df = api.alpha_vantage.historic_quotes(ticker, adjusted=True, output_format='pandas')
            df.columns = [f"{ticker} Open", f"{ticker} High", f"{ticker} Low", f"{ticker} Close", f"{ticker} Adjusted Close", f"{ticker} Volume", f"{ticker} Dividend Amount", f"{ticker} Split Coefficient"]
            tick_list.append(df)
        except:
            if ticker in assets:
                raise Exception('The ticker name is right, but AlphaVantage cannot pull its data, make sure the asset is tradeable')
            else:
                raise Exception('That is not a proper ticker symbol')
                
    data = pd.concat(tick_list, axis='columns', join='inner')
    return data
    

def data_clean(data):
    
    #drop unnecessary data
    data.drop([col for col in data.columns if 'Adjusted' in col], axis=1, inplace=True)
    data.drop([col for col in data.columns if 'Dividend' in col], axis=1, inplace=True)
    data.drop([col for col in data.columns if 'Split' in col], axis=1, inplace=True)
    data.drop([col for col in data.columns if 'High' in col], axis=1, inplace=True)
    data.drop([col for col in data.columns if 'Low' in col], axis=1, inplace=True)
    
    #rename the columns something that is actually understandable
    #data.columns = ['Ford Open','Ford Close','Ford Volume','Apple Open','Apple Close','Apple Volume']
    return data


In [6]:
api = load()
data = data_pull(api)
cleaned = data_clean(data)
cleaned.head()

C:\Users\andre\anaconda3\envs\newviznow\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Make sure your api keys are renamed to fit this function or alter the function to fit your names
  


,F Open,F Close,F Volume,AAPL Open,AAPL Close,AAPL Volume,ENPH Open,ENPH Close,ENPH Volume,WMT Open,WMT Close,WMT Volume,KO Open,KO Close,KO Volume
date,,,,,,,,,,,,,,,
2012-03-30,12.55,12.475,36381000.0,608.77,599.55,26108500.0,7.50,7.34,5048600.0,61.07,61.20,6941400.0,73.62,74.01,13277900.0
2012-04-02,12.50,12.620,43966900.0,601.83,618.63,21369700.0,7.65,7.81,495500.0,61.08,61.36,6465900.0,73.83,74.14,7289500.0
2012-04-03,12.71,12.640,97034200.0,627.30,629.32,29805700.0,7.75,7.64,316600.0,61.14,60.65,11180100.0,73.67,73.76,6494600.0
2012-04-04,12.46,12.505,51505000.0,624.35,624.31,20463600.0,7.77,7.45,165100.0,60.53,60.26,10851700.0,73.29,73.46,6243700.0
2012-04-05,12.38,12.470,37939500.0,626.98,633.68,22903500.0,7.48,7.43,104300.0,60.20,60.67,6528700.0,73.28,73.47,5765700.0


In [7]:
def strategy(cleaned):
    #Create a new dataframe of only the closing values
    df = cleaned.loc[:,[i for i in cleaned.columns if i.endswith('Close')]]

    #Rename the columns as only the ticker (just drop the word 'close')
    df.columns = df.columns.str.replace(' Close','')
    
    # Sort the date by ascending order so that the pct change function calculates properly
    df = df.sort_index(ascending=True)

    #Loop through the columns and create a percentage change column
    for col in df.columns:
        ticker = col

        df[f"{ticker} % Change"] = df[col].pct_change() * 100

    
    
#     init an entry signal as 0.0 to use as it loops over the data 
    df['Entry'] = 0.0
    df['Entry'] = np.where(df['F % Change'] <= -2.0, 1.0, 0.0)
    df['Exit'] = 0.0
    df['Exit'] = np.where(df['F % Change'] >= 2.0, 1.0, 0.0)
    df['Signal'] = df['Entry'] - df['Exit']
    return df

In [9]:
strat = strategy(cleaned)
#strat.sort_index(ascending=True)
strat.head()

,F,AAPL,ENPH,WMT,KO,F % Change,AAPL % Change,ENPH % Change,WMT % Change,KO % Change,Entry,Exit,Signal
date,,,,,,,,,,,,,
2012-03-30,12.475,599.55,7.34,61.20,74.01,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
2012-04-02,12.620,618.63,7.81,61.36,74.14,1.162325,3.182387,6.403270,0.261438,0.175652,0.0,0.0,0.0
2012-04-03,12.640,629.32,7.64,60.65,73.76,0.158479,1.728012,-2.176697,-1.157106,-0.512544,0.0,0.0,0.0
2012-04-04,12.505,624.31,7.45,60.26,73.46,-1.068038,-0.796097,-2.486911,-0.643034,-0.406725,0.0,0.0,0.0
2012-04-05,12.470,633.68,7.43,60.67,73.47,-0.279888,1.500857,-0.268456,0.680385,0.013613,0.0,0.0,0.0


In [10]:
def backtest(strat):
    backtest_df = strat[['F', 'F % Change', 'Entry', 'Exit', 'Signal']].copy()
    bt = backtest_df
    current_pos = 0
    cash = 100000
    bt['Current Position'] = current_pos
    bt['Current Cash'] = cash
    for i, row in bt.iterrows():

        if bt.at[i, 'Signal'] == 1 and current_pos == 0:
            current_pos += 1000
            cash = cash - (bt.at[i, 'F']*current_pos)
            bt.at[i, 'Current Position'] = current_pos
            bt.at[i, 'Current Cash'] = cash

        elif bt.at[i, 'Signal'] == 1 and current_pos == 1000:
            current_pos = current_pos
            cash = cash
            bt.at[i, 'Current Position'] = current_pos
            bt.at[i, 'Current Cash'] = cash

        elif bt.at[i, 'Signal'] == -1 and current_pos != 0:
            current_pos = 0
            cash = cash + (bt.at[i, 'F']*1000)
            bt.at[i, 'Current Position'] = current_pos
            bt.at[i, 'Current Cash'] = cash

        elif bt.at[i, 'Signal'] == -1 and current_pos == 0:
            current_pos = current_pos
            cash = cash
            bt.at[i, 'Current Position'] = current_pos
            bt.at[i, 'Current Cash'] = cash

        elif bt.at[i, 'Signal'] == 0 and current_pos != 0:
            current_pos = current_pos
            cash = cash
            bt.at[i, 'Current Position'] = current_pos
            bt.at[i, 'Current Cash'] = cash
        elif bt.at[i, 'Signal'] == 0 and current_pos == 0:
            current_pos = 0
            cash = cash
            bt.at[i, 'Current Position'] = current_pos
            bt.at[i, 'Current Cash'] = cash

    return backtest_df

In [12]:
bt = backtest(strat)
bt.head(25)

,F,F % Change,Entry,Exit,Signal,Current Position,Current Cash
date,,,,,,,
2012-03-30,12.475,NaN,0.0,0.0,0.0,0,100000
2012-04-02,12.620,1.162325,0.0,0.0,0.0,0,100000
2012-04-03,12.640,0.158479,0.0,0.0,0.0,0,100000
2012-04-04,12.505,-1.068038,0.0,0.0,0.0,0,100000
2012-04-05,12.470,-0.279888,0.0,0.0,0.0,0,100000
2012-04-09,12.220,-2.004812,1.0,0.0,1.0,1000,87780
2012-04-10,11.790,-3.518822,1.0,0.0,1.0,1000,87780
2012-04-11,11.910,1.017812,0.0,0.0,0.0,1000,87780
2012-04-12,12.070,1.343409,0.0,0.0,0.0,1000,87780
